In [83]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import acquire
import prepare

In [59]:
df = pd.read_csv('Levels_Fyi_Salary_Data.csv')

In [60]:
df.head()

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
0,6/7/2017 11:33:27,Oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.5,NaN,107000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
1,6/10/2017 17:11:29,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.0,NaN,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN
2,6/11/2017 14:53:57,Amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.0,NaN,155000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,6/17/2017 0:23:14,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,NaN,157000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
4,6/20/2017 10:58:51,Microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.0,NaN,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [61]:
df.shape

(62642, 29)

In [62]:
df.tag.value_counts()

Full Stack                        11382
Distributed Systems (Back-End)    10838
API Development (Back-End)         6277
ML / AI                            4204
Web Development (Front-End)        2971
                                  ...  
@fahed                                1
Embedded, generalist                  1
Generalist PSA                        1
Low-Level Integration                 1
TMT M&A                               1
Name: tag, Length: 3058, dtype: int64

In [63]:
df.isnull().sum()

timestamp                      0
company                        5
level                        119
title                          0
totalyearlycompensation        0
location                       0
yearsofexperience              0
yearsatcompany                 0
tag                          854
basesalary                     0
stockgrantvalue                0
bonus                          0
gender                     19540
otherdetails               22505
cityid                         0
dmaid                          2
rowNumber                      0
Masters_Degree                 0
Bachelors_Degree               0
Doctorate_Degree               0
Highschool                     0
Some_College                   0
Race_Asian                     0
Race_White                     0
Race_Two_Or_More               0
Race_Black                     0
Race_Hispanic                  0
Race                       40215
Education                  32272
dtype: int64

In [64]:
df.drop(columns = ['timestamp','Race','Education','otherdetails','gender','rowNumber'],inplace = True)

In [65]:
df.drop(columns = ['Race_Asian','Race_White','Race_Two_Or_More','Race_Black','Race_Hispanic'],inplace = True)

In [66]:
df.dropna(inplace = True)

In [67]:
df.columns

Index(['company', 'level', 'title', 'totalyearlycompensation', 'location',
       'yearsofexperience', 'yearsatcompany', 'tag', 'basesalary',
       'stockgrantvalue', 'bonus', 'cityid', 'dmaid', 'Masters_Degree',
       'Bachelors_Degree', 'Doctorate_Degree', 'Highschool', 'Some_College'],
      dtype='object')

In [68]:
df.isnull().sum()

company                    0
level                      0
title                      0
totalyearlycompensation    0
location                   0
yearsofexperience          0
yearsatcompany             0
tag                        0
basesalary                 0
stockgrantvalue            0
bonus                      0
cityid                     0
dmaid                      0
Masters_Degree             0
Bachelors_Degree           0
Doctorate_Degree           0
Highschool                 0
Some_College               0
dtype: int64

In [69]:
df.dropna(inplace = True)

In [70]:
df.isnull().sum()

company                    0
level                      0
title                      0
totalyearlycompensation    0
location                   0
yearsofexperience          0
yearsatcompany             0
tag                        0
basesalary                 0
stockgrantvalue            0
bonus                      0
cityid                     0
dmaid                      0
Masters_Degree             0
Bachelors_Degree           0
Doctorate_Degree           0
Highschool                 0
Some_College               0
dtype: int64

In [71]:
df.head()

,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,cityid,dmaid,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College
409,Yahoo,IC2,Software Engineer,160500,"Sunnyvale, CA",0.58,0.58,Full Stack,128000.0,13000.0,19200.0,7472,807.0,0,0,0,0,0
410,Oracle,M3,Software Engineering Manager,340000,"Redwood City, CA",16.00,15.00,Distributed Systems (Back-End),206000.0,134000.0,0.0,7392,807.0,0,0,0,0,0
411,Qualcomm,Senior Staff Engineer,Software Engineer,240000,"Santa Clara, CA",15.00,6.00,Embedded Systems,0.0,0.0,0.0,7434,807.0,0,0,0,0,0
412,Splunk,P2,Software Engineer,160000,"Seattle, WA",5.00,3.00,Full Stack,0.0,0.0,0.0,11527,819.0,0,0,0,0,0
413,Google,L3,Software Engineer,208000,"Mountain View, CA",0.00,0.00,Distributed Systems (Back-End),116000.0,65000.0,17000.0,7322,807.0,0,0,0,0,0


In [73]:
df.Highschool.value_counts()

0    61346
1      319
Name: Highschool, dtype: int64

In [74]:
df.Some_College.value_counts()

0    61310
1      355
Name: Some_College, dtype: int64

In [76]:
df.Bachelors_Degree.value_counts()

0    49097
1    12568
Name: Bachelors_Degree, dtype: int64

In [77]:
df.Masters_Degree.value_counts()

0    46324
1    15341
Name: Masters_Degree, dtype: int64

In [78]:
df.Doctorate_Degree.value_counts()

0    59867
1     1798
Name: Doctorate_Degree, dtype: int64

In [80]:
df = df[df.Highschool + df.Some_College + df.Bachelors_Degree + df.Masters_Degree + df.Doctorate_Degree > 0]

In [81]:
df.shape

(30277, 18)

In [82]:
df.head()

,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,cityid,dmaid,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College
717,Facebook,E4,Software Engineer,239000,"Menlo Park, CA",4.0,0.0,ML / AI,140000.0,60000.0,39000.0,7300,807.0,0,0,1,0,0
3427,Capital One,Senior Associate,Data Scientist,125000,"Washington, DC",1.0,0.0,ML / AI,121000.0,0.0,4000.0,40303,511.0,0,0,1,0,0
5037,Bloomberg,Software Engineer,Software Engineer,170000,"New York, NY",0.0,0.0,Distributed Systems (Back-End),142000.0,0.0,18000.0,10182,501.0,1,0,0,0,0
5392,Amazon,L5,Data Scientist,332000,"Palo Alto, CA",1.0,0.0,Optimization,185000.0,17000.0,130000.0,7351,807.0,0,0,1,0,0
6055,Apple,ICT3,Software Engineer,240000,"Cupertino, CA",3.0,3.0,ML / AI,150000.0,75000.0,15000.0,7158,807.0,0,0,1,0,0


In [84]:
df.columns

Index(['company', 'level', 'title', 'totalyearlycompensation', 'location',
       'yearsofexperience', 'yearsatcompany', 'tag', 'basesalary',
       'stockgrantvalue', 'bonus', 'cityid', 'dmaid', 'Masters_Degree',
       'Bachelors_Degree', 'Doctorate_Degree', 'Highschool', 'Some_College'],
      dtype='object')

In [85]:
df['highly_experenced'] = df.yearsofexperience >= 4

<ipython-input-85-b86898b8b741>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['highly_experenced'] = df.yearsofexperience >= 4


In [86]:
df.head()

,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,cityid,dmaid,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College,highly_experenced
717,Facebook,E4,Software Engineer,239000,"Menlo Park, CA",4.0,0.0,ML / AI,140000.0,60000.0,39000.0,7300,807.0,0,0,1,0,0,True
3427,Capital One,Senior Associate,Data Scientist,125000,"Washington, DC",1.0,0.0,ML / AI,121000.0,0.0,4000.0,40303,511.0,0,0,1,0,0,False
5037,Bloomberg,Software Engineer,Software Engineer,170000,"New York, NY",0.0,0.0,Distributed Systems (Back-End),142000.0,0.0,18000.0,10182,501.0,1,0,0,0,0,False
5392,Amazon,L5,Data Scientist,332000,"Palo Alto, CA",1.0,0.0,Optimization,185000.0,17000.0,130000.0,7351,807.0,0,0,1,0,0,False
6055,Apple,ICT3,Software Engineer,240000,"Cupertino, CA",3.0,3.0,ML / AI,150000.0,75000.0,15000.0,7158,807.0,0,0,1,0,0,False
